# HyperLedger Indy

### Introduction:

<img src='./Images/HL-Indy.png'> ![](./Images/HL-Indy.png)

The most common (and universally hated) system, that of user IDs and passwords, is fraught with problems. The resulting lack of certainty of who is on the other keyboard has led to the loss of billions from hacks, data breaches, identity theft, scams and more. Further, that same lack of certainty has made many types of business transactions on the Internet impossible - the risk is just too high.

Hyperledger Indy has been created to add an identity layer to the Internet using a mechanism that is easy to use, enables online trust, and enhances privacy. 

#### What is identity on Internet?

The problems with identity on the Internet today come down to a single word - **trust**. When you are interacting online with someone, do you trust:


            Is the person you are connecting with online who they say they are?

            Are the claims they are making true?


### Type of Blockchain

    As we covered earlier in this course, there are several different types of blockchain systems characterized along two dimensions:
        access and validation. 
   **Bitcoin and Ethereum** are public *permissionless networks* - anyone can access them (public) and anyone can participate in the validation process (permissionless).
    
   The Hyperledger frameworks **(Fabric, Sawtooth, Iroha and Burrow)** are (primarily) used for *private/permissioned networks*, limited to who can access them (private) and who can participate in the validation process (permissioned).
    
<img src = "Images/PublicPermissionBlockchain.png"> ![](Images/PublicPermissionBlockchain.png)

### Blockchain Consensus Algorithm

Hyperledger Indy uses a consensus algorithm to decide the contents of the next block added to the chain. Specifically, Indy uses **Plenum**, an implementation of the **Redundant Byzantine Fault Tolerance (RBFT) algorithm**. Plenum achieves similar performance to other BFT algorithms in ideal conditions (no faulty participants), but its performance degrades far less than other algorithms (on the order of 3% versus up to 78% for others) when faults occur in the network.

Indy also implements a novel deployment of *Stewards - the nodes of the network that have permission to participate in the validation process.*

<img src = "Images/sovrinValidatorObserverNodes.png"> ![](Images/sovrinValidatorObserverNodes.png)

The **Sovrin Foundation** is a global non-profit that has deployed the Hyperledger Indy code on a (growing) number of nodes to create a running Public Permissioned Hyperledger Indy Blockchain instance. 


### Running Hyperledger Indy

In this section, we’ll walk through an example of running an Indy ecosystem on Docker - an instance of the Indy public ledger and a series of Indy Agents that can message one another and share Verifiable Credentials. 

There are three types of transactions:
    
            1. DIDs
    Those are the decentralized identifiers created by each participant in the demo (Alice, Faber, etc.). Look for entries with "role" and "verkey" in the data. You might even recognize the DIDs on the ledger with the DIDs on the demo web pages.
    
            2. Schema        
    Entries that define the data elements (the claims) that will be part of a Verifiable Credential. Look for the name of the schema, such as "Government-ID".
    
            3. Credential definitions
    Entries that are created by a credential issuer (like Faber) using a specific schema and (optionally, and not in this demo) a revocation registry. Credential definitions have a private key for each claim (and more), so look for entries with big long strings and references to schema claim names, e.g. "tax_id".
    
            4. Revocation registries
    Entries that give an issuer a way to revoke issued credentials in a non-correlatable way. In this demo, we are not using revocation capabilities, so you won’t find any on the ledger.

### The Hyperledger Indy GitHub Repos

As of August 2018, there were nine official Hyperledger Indy GitHub repos. To check the current status of the repos that make up Hyperledger Indy, go to the Hyperledger Indy Project Wiki page. Of those nine, six are important to describe here:

            indy-node
            indy-sdk
            indy-plenum, indy-crypto
            indy-agent
            indy-hipe.

We will discuss them in more detail on the next pages.

The other three are not (currently) particularly important to the general community.

#### Indy - node

Indy-node is (mostly) the Python code that implements the Indy public ledger - the set of permissioned instances (nodes) that accept and process public ledger read and write requests from Indy clients (usually called Agents).

#### Indy - sdk

The indy-sdk repo contains the code that allows a piece of software - an Agent - to interact with the public ledger and to keep track (in a Wallet - a specialized data store) of the keys and other identity-related data. The indy-sdk consists of one core component (written mostly in Rust) that compiles to a “c-callable” library (called libindy) and a number of language specific "wrappers" for the library. A “c-callable” library means that the library components can be called from the majority of languages, eliminating the need for an implementation in each language. The wrappers, available (as of September 2018) in Python, Java, C# (.NET) and Node.js, allow the creation of a Indy agents in each of those languages.

As mentioned, the indy-sdk implements a Wallet that is used to store the Indy data collected by an identity on an Indy network - DIDs, private keys, Verifiable Credentials and more. The indy-sdk implements a default implementation of the Wallet, and is pluggable - meaning a capable developer can implement a replacement Wallet. The default Indy Wallet is based on the open source SQLite database.

#### indy-hipe

The indy-hipe repo is a set of "Hyperledger Indy Project Enhancements" (you know, HIPEs) - documents that describe Hyperledger Indy features, approaches to interoperability and other things that have or will make Indy better. It’s a great place to learn the details of some of the key features of Indy: the Wallet implementation, revocation and more.



### General Information

This doc is about supported transactions and their representation on the Ledger (that is, the internal one). If you are
interested in the format of a client’s request (both write and read), then have a look at requests.
    
    • All transactions are stored in a distributed ledger (replicated on all nodes)
    
    • The ledger is based on a Merkle Tree
    
    • The ledger consists of two things:
        – transactions log as a sequence of key-value pairs where key is a sequence number of the transaction and 
        value is the serialized transaction.
        – merkle tree (where hashes for leaves and nodes are persisted)
    
    • Each transaction has a sequence number (no gaps) - keys in transactions log
    
    • So, this can be considered a blockchain where each block’s size is equal to 1
    
    • There are multiple ledgers by default:
        – pool ledger: transactions related to pool/network configuration (listing all nodes, their keys and addresses)
        – config ledger: transactions for pool configuration plus transactions related to pool upgrade
        – domain ledger: all main domain and application specific transactions (including NYM transactions for DID)
    
    • All transactions are serialized to MsgPack format
    
    • All transactions (both transaction log and merkle tree hash stores) are stored in a LevelDB
    
    • One can use the read_ledger script to get transactions for a specified ledger in a readable format (JSON)
    
    • See roles and permissions for a list of roles and they type of transactions they can create.

### Genesis Transactions

As Indy is a public permissioned blockchain, each ledger may have a number of pre-defined transactions defining the
initial pool and network.
    
    • pool genesis transactions define initial trusted nodes in the pool
    
    • domain genesis transactions define initial trusted trustees and stewards
    
### Common Structure

Each transaction has the following structure consisting of metadata values (common for all transaction types) and
transaction specific data:

ModuleNotFoundError: No module named 'indy'